In [3]:
import pandas as pd # import pandas for dataframes
import requests
from bs4 import BeautifulSoup
import locale
from datetime import datetime

In [20]:
def initiate_scraper():
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko'}
    return headers

In [21]:
headers = initiate_scraper()

In [22]:
def get_seeallreviews_link(url):
    seeallreviews_HTML = requests.get(url=url)
    seeall_soup = BeautifulSoup(seeallreviews_HTML.content, features = 'lxml')
    seeallreviews_link = seeall_soup.find_all("a", {"data-hook":"see-all-reviews-link-foot"})
    try:
        seeallreviews_link = seeallreviews_link[0].get('href')
        domain = 'https://www.amazon.com'
        seeallreviews_link = str(domain) + str(seeallreviews_link)
    except:
        seeallreviews_link = print("Hmmm... seems like Amazon has blocked the IP... Sorry about that! Hope you come back later (:")
    return seeallreviews_link
    print(seeallreviews_link)

In [23]:
url = 'https://www.amazon.com/dp/B07PF1Y28C?ref=MarsFS_AUCC_cn'
seeallreviews_HTML = requests.get(url, headers = headers)
seeall_soup = BeautifulSoup(seeallreviews_HTML.content, features = 'lxml')

In [24]:
url = 'https://www.amazon.com/dp/B07PF1Y28C?ref=MarsFS_AUCC_cn'
see = get_seeallreviews_link(url)
see

Hmmm... seems like Amazon has blocked the IP... Sorry about that! Hope you come back later (:


In [25]:
def get_numberofreviews(seeallreviews_link):
    # Get the HTML code for the first page (not the 'see all reviews site')
    firstpage_HTML = requests.get(seeallreviews_link, headers = headers)
    # Create a soup from HTML
    firstpage_soup = BeautifulSoup(firstpage_HTML.content, features = 'lxml')
    # Find chunk of code that includes total number of customer reviews
    # Get text, which includes the number of customer reviews
    numrev_subsoup = firstpage_soup.find("div", {'data-hook':'total-review-count'})
    numrev_text = numrev_subsoup[0].get_text()
    
    numrev = str(numrev_text)
    # There are 18 characters in the string ' customer ratings', so we can backwards slice to get the number and then raplace any commas with nothing
    numberof_reviews = numrev.strip()[:-18]
    numberof_reviews = numberof_reviews.replace(',','')
    return numberof_reviews

In [26]:
def get_numberof_pages(numberof_reviews):
    n = (round(float(numberof_reviews)/10,0))-3
    return n

In [27]:
def get_soup(page_link):
    response_page = requests.get(page_link, headers = headers)
    soup_page = BeautifulSoup(response_page.content, features = 'lxml')
    return soup_page

In [28]:
def get_next_page(current_soup):
    domain = 'https://www.amazon.com'
    paging_link = current_soup.find_all("li", {'class':'a-last'})
    paging_link = paging_link[0].find("a")['href']
    paging_link = str(domain) + str(paging_link)
    return paging_link

In [30]:
url = 'https://www.amazon.com/dp/B07PF1Y28C?ref=MarsFS_AUCC_cn'
url = get_seeallreviews_link(url)
current_soup = get_soup(url)
get_next_page(current_soup)

Hmmm... seems like Amazon has blocked the IP... Sorry about that! Hope you come back later (:


MissingSchema: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?

In [1]:
def get_page_reviews(page_soup):
    page_reviews = page_soup.find_all("div", {'data-hook':'review'}) # create soup of first page
    page_reviews_df = pd.DataFrame()
    for i in range(10):
        page_reviewsi = page_reviews[i]
        name = page_reviewsi.find_all("span", {'class':'a-profile-name'})[0].get_text()
        date = page_reviewsi.find_all("span", {'data-hook':'review-date'})[0].get_text()
        date = date[33:]
        date = datetime.strptime(str(date), "%B %d, %Y")
        stars = page_reviewsi.find_all("a", {'class':'a-link-normal'})[0].get_text()
        titlerev = page_reviewsi.find_all("a", {'data-hook':'review-title'})[0].get_text(strip=True)
        review_body = page_reviewsi.find_all("span", {'data-hook':'review-body'})[0].get_text(strip=True)
        try:
            helpful = page_reviewsi.find_all("span", {'data-hook':'helpful-vote-statement'})[0].get_text()
            helpful = helpful[:-26]
        except:
            helpful = '0 people found this helpful'
        row_df = pd.DataFrame({'name':[name], 'date': [date], 'stars': [stars], 'title':[titlerev], 'review_body':[review_body], 'helpful':[helpful]})
        page_reviews_df = page_reviews_df.append(row_df)
        page_reviews_df = pd.DataFrame(page_reviews_df)
    return page_reviews_df


In [35]:
def get_all_reviews(first_page_link, total_pages):
    urls_list = []
    all_reviews = pd.DataFrame()
    domain = 'https://www.amazon.com'
    
    soup_pagei = get_soup(first_page_link)
        
    for i in range(min(int(total_pages), 499)):
        try:
            
            page_reviews_df = get_page_reviews(soup_pagei)
        
            urls_list.append(first_page_link)
        
            all_reviews = all_reviews.append(page_reviews_df, ignore_index = True)

            first_page_link = get_next_page(soup_pagei)
        
            soup_pagei = get_soup(first_page_link)
        except:
            break
        print(i)
    return all_reviews

In [36]:
url = 'https://www.amazon.com/dp/B07MLY3JKV?ref=ODS_v2_FS_AUCC_cst'
headers = initiate_scraper()
try:
    seeallreviews_link = get_seeallreviews_link(url)
except:
    seeallreviews_link = url
    pass
numberof_reviews = get_numberofreviews(seeallreviews_link)
numberof_pages = get_numberof_pages(numberof_reviews)

page_reviews = get_all_reviews(seeallreviews_link, numberof_pages-2)
page_reviews

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,name,date,stars,title,review_body,helpful
0,Joseph Siegler,2019-11-19,VINE VOICE,Very small footprint Echo - size impacts audio...,I got this for one of the primary reasons I su...,432 people found this helpful
1,C.Q. Young,2019-11-20,5.0 out of 5 stars,It is an Alexa enabled device that you can jus...,I use it outside in my caport to control all m...,310 people found this helpful
2,woodstocksb,2019-11-18,1.0 out of 5 stars,Sound so bad,"If it did not have a speaker, I would have rat...",216 people found this helpful
3,Sonnig Freitag,2019-11-18,5.0 out of 5 stars,Another great Echo device,Wow! They are finally letting me review this....,170 people found this helpful
4,TXEggplant,2019-11-20,4.0 out of 5 stars,Not My Favorite Echo Device but Not My Least F...,I'm a HUGE fan of Echo devices. I have one in ...,132 people found this helpful
...,...,...,...,...,...,...
3245,Frank,2020-01-12,5.0 out of 5 stars,Works well!,Works very well,0 people found this helpful
3246,chaz Croneberger,2020-01-20,5.0 out of 5 stars,Simple easy setup works great,Great product,0 people found this helpful
3247,Amazon Customer,2020-01-11,5.0 out of 5 stars,Works great,Works great,0 people found this helpful
3248,Nancy,2020-02-02,5.0 out of 5 stars,Good,Good,0 people found this helpful


In [37]:
def get_dataframe(url):
    headers = initiate_scraper()
    try:
        seeallreviews_link = get_seeallreviews_link(url)
    except:
        seeallreviews_link = url
        pass
    numberof_reviews = get_numberofreviews(seeallreviews_link)
    numberof_pages = get_numberof_pages(numberof_reviews)
    page_reviews = get_all_reviews(seeallreviews_link, numberof_pages-2)
    return page_reviews
  

In [26]:
page_reviews = get_dataframe('https://www.amazon.com/gp/product/B07Z8BMLY8?pf_rd_r=TSWM40DT3THWKKJKY6S3&pf_rd_p=ab873d20-a0ca-439b-ac45-cd78f07a84d8&th=1')
page_reviews

0
1


,name,date,stars,title,review_body,helpful
0,Jones,2018-11-02,3.0 out of 5 stars,Bait and switch!,I am disappointed with these bars I tried the ...,56 people found this helpful
1,Olive,2018-06-14,3.0 out of 5 stars,taste and texture don't do it for me..,I eat protein bars as breakfast replacements a...,59 people found this helpful
2,John B. Alleman,2018-03-06,5.0 out of 5 stars,They don't taste like dirty hippies.,OMG!!! It's actually good! I have tried just...,67 people found this helpful
3,Peter Demarco,2018-02-23,5.0 out of 5 stars,I feel like they hit the right balance between...,Because I've been watching the Winter Olympics...,46 people found this helpful
4,K. McCulloch,2018-06-13,5.0 out of 5 stars,"Taste good, don't make me sick",These are my favorites. I buy them over and ov...,30 people found this helpful
5,Maddy Schier,2018-01-03,5.0 out of 5 stars,ONLY BAR I'LL EAT,I honestly thought I couldn't like a protein b...,24 people found this helpful
6,Tyler Sullivan,2018-01-28,5.0 out of 5 stars,Great Tasting and Very Filling,I originally bought the atlas bar for its high...,31 people found this helpful
7,Zeejet,2018-01-16,5.0 out of 5 stars,Awesome protein bars! Tastes great (amazing te...,A fantastic protein bar with natural and minim...,18 people found this helpful
8,Dean Jones Jr.,2019-12-18,3.0 out of 5 stars,Not as good as it used to be...,These used to be pretty tasty. It used to tast...,6 people found this helpful
9,Brett N,2019-03-29,5.0 out of 5 stars,Addicted to Atlas Bars,"I'm fully addicted to these bars, particularly...",9 people found this helpful


In [23]:
page_reviews.iloc[0]['review_body']

'I am disappointed with these bars I tried the vanilla almond flavor and it says on the box really clearly "grass-fed whey" so I assumed that\'s what it was made out of, but when I put my glasses on and looked at the ingredients I found it also has whey protein isolate and milk protein isolate which is not grass-fed or even organic.  I don\'t think it\'s right that they list grass-fed whey on the box knowing that that\'s what people want and that you have to put your glasses on to find out that it\'s actually got non grass-fed whey protein isolate and milk protein isolate. The bar does taste good though, if the the source of your protein doesn\'t matter to you.'

In [24]:
page_reviews

,name,date,stars,title,review_body
0,Jones,2018-11-02,3.0 out of 5 stars,Bait and switch!,I am disappointed with these bars I tried the ...
1,Olive,2018-06-14,3.0 out of 5 stars,taste and texture don't do it for me..,I eat protein bars as breakfast replacements a...
2,John B. Alleman,2018-03-06,5.0 out of 5 stars,They don't taste like dirty hippies.,OMG!!! It's actually good! I have tried just...
3,Peter Demarco,2018-02-23,5.0 out of 5 stars,I feel like they hit the right balance between...,Because I've been watching the Winter Olympics...
4,K. McCulloch,2018-06-13,5.0 out of 5 stars,"Taste good, don't make me sick",These are my favorites. I buy them over and ov...
...,...,...,...,...,...
475,S.Messex,2018-03-24,3.0 out of 5 stars,Great tase,"I purchased the vanilla almond bars, and the f..."
476,MNIAMO,2018-09-14,5.0 out of 5 stars,My favorite protein bar of all time,There are really only two periods of my life: ...
477,Amazon Customer,2018-12-18,2.0 out of 5 stars,Taste is poor and texture is odd,I am not a person who likes things to be overl...
478,Amazon Customer,2018-10-14,3.0 out of 5 stars,Not the best tasting,I tried these based on other reviews but havin...
